In [0]:
# -40.900557, 174.885971, New Zealand
import sys
import csv
import numpy as np

country = []
latitude = []
longtitude = []

with open('country.csv', newline='') as csvfile:
  spamreader = csv.reader(csvfile, delimiter=' ', quotechar='|')
  for row in spamreader:
    items = row[0].split(',')
    country.append(items[2])
    latitude.append( float(items[0]))
    longtitude.append( float(items[1]) )

In [0]:
latitude = np.array(latitude)
longtitude = np.array(longtitude)

In [0]:
lt_mean = latitude.mean()
lt_std = latitude.std()

latitude = (latitude - lt_mean)/lt_std

lo_mean = longtitude.mean()
lo_std = longtitude.std()

longtitude = (longtitude - lo_mean)/lo_std

In [0]:
import string

characters = string.printable
token_index = dict(zip(characters, range(1, len(characters) +1 )))

max_length = 13

results = np.zeros((len(country), max_length, max(token_index.values()) + 1))
for i, sample in enumerate(country):
  for j, character in enumerate(sample):
    index = token_index.get(character)
    results[i, j , index] = 1.

In [129]:
np.shape(results)
print(np.shape(results))
print(type(results))
print(type(results[0][0][0]))

(243, 13, 101)
<class 'numpy.ndarray'>
<class 'numpy.float64'>


In [0]:
from keras import layers
from keras import Input
from keras.models import Model

countryInput = Input(shape=(13 ,101, ), dtype='float32', name='position')
print(np.shape(countryInput))

x = None
x = layers.Conv1D(64, 5, activation='relu')(countryInput)
x = layers.MaxPooling1D(3)(x)
x = layers.Conv1D(64, 3, activation='relu')(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(16, activation='relu')(x)

latitude_prediction = layers.Dense(1, name='latitude')(x)
longtitude_prediction = layers.Dense(1, name='longtitude')(x)

model = Model(countryInput, [latitude_prediction, longtitude_prediction])

model.compile(optimizer='rmsprop', loss=['mse', 'mse'], loss_weights=[1., 1.])

model.fit(results, [latitude, longtitude], epochs=250, batch_size=16)

In [0]:
import string


max_length = 13

target = ['New Zealand']

test_results = np.zeros((len(target), max_length, max(token_index.values()) + 1))
for i, sample in enumerate(target):
  for j, character in enumerate(sample):
    index = token_index.get(character)
    test_results[i, j , index] = 1.

In [132]:
c = model.predict(test_results)
print(c)

[array([[-0.37207735]], dtype=float32), array([[0.6565605]], dtype=float32)]


In [133]:
print(c[0][0][0]*lt_std+lt_mean, c[1][0][0]*lo_std+lo_mean)

6.523940594256411 60.71866637946135
